# Network and Geospatial Analyses
## Part 1: Geospatial Analysis
`Author: Stijn Overmeen (stijn.overmeen@nelen-schuurmans.nl)`

Assumed knowledge:
  - Pizzacursus netwerk en GEO-analyses in Python - Intro

This tutorial consists of a some additional theory and practice code. After this, you are ready to start working on the first case of the day:
  - [Theory: splitting geometries](#theory1)
  - [Theory: finding nearest feature](#theory2)
  - [Case 1 - Afstand-tot-Koelte: preprocessing the network](#case1)
  - [Bonus - Slicing polygons in half](#bonus)
   
#### Disclaimer: This course material is intended solely for internal use within Nelen & Schuurmans and is provided exclusively for educational purposes. All rights, including copyright, pertaining to this material are owned or licensed by Nelen & Schuurmans.  

<a id='theory1'></a>
### Theory: splitting geometries
Splitting geometries into smaller parts, while essential in some analyses, can introduce potential dangers. Altering the geometry may have unintended consequences, such as introducing topological errors or misrepresenting real-world features.

Let's start splitting geometries, while being careful that we do not introduce toplogical errors.

In [ ]:
# We load in the Python packages we learned to use in the Intro
from pathlib import Path
from tqdm import tqdm
from shapely import Point, LineString, Polygon, make_valid
import geopandas as gpd

# We also need these (basic) Python packages
import numpy as np
import matplotlib.pyplot as plt

#### Splitting lines in smaller segments
Let's start with splitting lines in smaller segments. Should be quite straightforward, right?

##### split_line_by_length_1
I wrote the function `split_line_by_length_1`. It checks whether the original line is equal or smaller than the maximum length. If so, it returns the original line, it does not need to be splitted. 

Else, we start the splitting procedure. We first decide how many segments we need. We do this based on the total length divided by the segment length. We take the ceil. The ceil of x is the smallest integer i, such that i >= x. In other words, if we need 4.2 segments, we will create 5.

Then we will calculate the length of each segment based on the number of segments. We retrieve the coordinates on the line belonging to each segment with its new length in a loop. In another loop we convert the coordinates to a LineString and append the new line to the list of segments. 

Makes sense?

In [ ]:
def split_line_by_length_1(line_geometry, max_segment_length):
    """ check if line is longer than threshold and if so, split into smaller segments"""
    total_length = line_geometry.length
    if total_length <= max_segment_length:
        return [line_geometry]
    
    segments = []
    
    num_segments = int(np.ceil(total_length / max_segment_length))
    step = total_length / num_segments

    split_distances = [i * step for i in range(num_segments + 1)]
    split_points = [line_geometry.interpolate(distance) for distance in split_distances]

    for i in range(len(split_points) - 1):
        segment = LineString([split_points[i], split_points[i + 1]])
        segments.append(segment)

    return segments

Let's put this function to the test.

In [ ]:
# Example
line_coords = [(0, 0), (2.1, 5.77), (11.53, 12.74), (19.9, 11.6), (22.22, 9.96)]
line_geometry = LineString(line_coords)
max_segment_length = 5.0

In [ ]:
# Let's run our splitting function
resulting_segments_1 = split_line_by_length_1(line_geometry, max_segment_length)

In [ ]:
# Print segment lengths
for segment in resulting_segments_1:
    print(segment.length)

In [ ]:
# Function to plot the LineString and its segments
def plot_line_and_segments(line_geometry, segments):
    fig, ax = plt.subplots()

    # Plot the original LineString
    ax.plot(*line_geometry.xy, marker='o', label='Original LineString')

    # Plot the segments
    for segment in segments:
        ax.plot(*segment.xy, marker='x', linestyle='dashed', label='Segment')

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.legend()
    plt.show()

In [ ]:
# Let's create the plot
plot_line_and_segments(line_geometry, resulting_segments_1)

Hmm, take some time to analyse this result. We ended up changing the geometry. Why do you think so? What is "wrong" with our approach?

...

##### split_line_by_length_2
I wrote the function `split_line_by_length_2`. Take a look at the function description and the code itself. What has changed?

...

In [ ]:
def split_line_by_length_2(line_geometry, max_segment_length):
    """ 
    iterate over line coordinates (the original line segments)
    check if each line segment is longer than threshold and if so, split segment into smaller subsegments
    """
    coords = list(line_geometry.coords)
    segments = []

    for i in range(len(coords) - 1):
        start = Point(coords[i])
        end = Point(coords[i + 1])
        segment_length = start.distance(end)

        if segment_length <= max_segment_length:
            segments.append(LineString([start, end]))
        else:
            num_subsegments = int(np.ceil(segment_length / max_segment_length))
            x_coords = np.linspace(start.x, end.x, num_subsegments + 1)
            y_coords = np.linspace(start.y, end.y, num_subsegments + 1)

            subsegment_points = [Point(x, y) for x, y in zip(x_coords, y_coords)]
            segments.extend([LineString([subsegment_points[j], subsegment_points[j + 1]]) for j in range(len(subsegment_points) - 1)])

    return segments

In [ ]:
# Let's run the second splitting function and check out the results
resulting_segments_2 = split_line_by_length_2(line_geometry, max_segment_length)

for segment in resulting_segments_2:
    print(segment.length)

In [ ]:
plot_line_and_segments(line_geometry, resulting_segments_2)

Are you happy with this result? Play around with the input line geometry or splitting distance if you wish to see if this function works for all cases.

#### Check geometry validity before any analysis
Polygons are complex geometries. Sometimes, an invalid polygon geometry is provided by the client or created by a GIS module. This could give issues within your analysis. A polygon can be invalid due to different reasons, for instance:
1. Self-Intersecting Polygons: A self-intersecting polygon is a polygon whose edges cross over each other, creating intersections within the polygon itself. Such polygons do not have a clear distinction between the interior and exterior areas, which can cause problems in spatial calculations and analyses.
2. Unclosed Polygons: Polygons must be closed loops, meaning that the first and last points in the polygon's coordinate sequence must be the same.

To address these problems, it's essential to perform data validation and cleaning before conducting spatial analyses. `shapely` and `geopandas` provide functions to identify and fix invalid geometries.

We start with loading in the polygons saved in the folder `material/part 1`, as `Polygons.gpkg` with layername `polygons`.

In [ ]:
# Load in the polygons, using the knowledge you've gained from the Intro
...

In [ ]:
# Check out the geodataframe
...

Next up, we check out these polygons by creating some plots.

In [ ]:
# Function to plot polygon
def plot_polygon(polygon_geometry):
    fig, ax = plt.subplots()
    x, y = polygon_geometry.exterior.coords.xy
    ax.plot(x, y)
    ax.fill(x, y, alpha=0.3)
    plt.show()

In [ ]:
# Plot Polygon J
polygon_geometry = polygons[polygons['ident'] == 'Polygon J'].iloc[0].geometry
plot_polygon(polygon_geometry)

In [ ]:
# Function to validate geometries and raise an Exception
def check_validity_geometry(geometry):
    if not geometry.is_valid:
        raise ValueError(f"Invalid geometry: {geometry}")

In [ ]:
# Check the validity of all polygons
...

Identify the invalid polygon. What is the polygon `ident`?

In [ ]:
# Identify the invalid polygon by filling in the correct ident
invalid_polygon = polygons[polygons["ident"] == "___"]

An alternative approach is not only checking whether the polygon is invalid, but fixing invalid polygons right away if possible.

In [ ]:
# Function to validate geometries and fix it if possible
def make_valid_geometry(geometry):
    if not geometry.is_valid:
        make_valid_geometry = make_valid(geometry)
        if not make_valid_geometry.is_valid:
            raise ValueError(f"We were unable to make this invalid geometry valid: {geometry}")
        return make_valid_geometry
    return geometry

In [ ]:
# Make the invalid geometries valid
valid_polygons = polygons.copy()

for idx, row in polygons.iterrows():
    valid_polygons.at[idx, 'geometry'] = make_valid_geometry(row.geometry)

In [ ]:
# Check out the geometry of the once invalid polygon to identify the difference
once_invalid_polygon = valid_polygons[valid_polygons["ident"] == "___"]

The make_valid module turned the geometry type into a MultiPolygon in order to create a valid geometry.

In [ ]:
# Convert MultiPolygon to a list of individual polygons
polygons_geometry_list = list(once_invalid_polygon.iloc[0].geometry.geoms)

In [ ]:
# Plot the polygon geometries in the list
...

A way to convert multi geometries into single geometries in a GeoDataFrame is by using built-in command `explode`.

In [ ]:
# Explode the MultiPolygons into individual Polygons
exploded_valid_polygons = valid_polygons.explode(index_parts=False)

Now we are confident we can start with our spatial manipulation.

<a id='theory2'></a>
### Theory: finding nearest feature



Writing a function to find the nearest feature of a geodataframe to an input feature of another geodataframe is not the most difficult function to write. 

- We should loop over all features and check the distance between the input feature and this feature. 
- We should keep track of the minimum distance and nearest feature by updating it in case a closer feature is found than the current nearest feature. 
- In the end, after the loop, we found the nearest feature.

In [ ]:
# Function to find the nearest feature of a geodataframe
def find_nearest(input_feature, search_gdf):
    nearest_feature = None
    min_distance = float('inf')
    for idx, feature in search_gdf.iterrows():
        distance = input_feature.distance(feature.geometry)
        if distance < min_distance:
            min_distance = distance
            nearest_feature = feature
    return nearest_feature

Let's try this function out. Please load the following layers from the folder `material/part 1`:
1. `Dummy Data.gpkg` with layername `buildings`.
2. `Dummy Data.gpkg` with layername `roads`.

In [ ]:
...

How many buildings and how many roads do we have?

In [ ]:
...

After loading in the buildings and roads, we try to find the nearest road to all buildings.

In [ ]:
# Let's find the nearest roads
building_road_analysis = {}
for idx, building in tqdm(buildings.iterrows(), total=len(buildings), desc="Finding nearest", unit="buildings"): 
    nearest_road = find_nearest(building.geometry, roads)
    building_road_analysis[building.gid] = {"nearest_road_id" : nearest_road.osm_id}

In [ ]:
building_road_analysis

Works fine, right? However, noticed the wait? Looping over all features in the geodataframe can be quite computational expensive.

#### Optimize performance with spatial indexing
Spatial indexing is a technique that optimizes spatial queries and operations on geographic data. It works by organizing geographic objects into a hierarchical structure, making searches faster and more efficient. When working with large datasets, spatial indexing is a must!

Spatial indexing, like the `R-tree` method integrated in the package `geopandas`, encloses objects within bounding rectangles and organizes them hierarchically. When you query, it selectively traverses this hierarchy, leading to rapid results.

In other words, we do the same analysis for way less features. A candidate, in this context, refers to a geographic feature from a dataset that might be the nearest neighbor to the input feature, based on spatial proximity.

Let's transform the function to make it use spatial indexing:

In [ ]:
# Function to find the nearest feature of a geodataframe using spatial indexing
def find_nearest_with_index(input_feature, search_gdf):   
    search_gdf_index = search_gdf.sindex # create the spatial index 
    # Use the spatial index to find the nearest feature
    
    possible_matches_index = list(search_gdf.sindex.intersection(input_feature.bounds))
    if not possible_matches_index:
        return find_nearest(input_feature, search_gdf) # if we do not find possible matches we return to the original method

    nearest_feature = None
    min_distance = float('inf')

    for idx in possible_matches_index:
        candidate = search_gdf.loc[idx]
        distance = input_feature.distance(candidate.geometry)
        if distance < min_distance:
            min_distance = distance
            nearest_feature = candidate

    return nearest_feature

In [ ]:
# Let's find the nearest roads using spatial index
building_road_analysis = {}
for idx, building in tqdm(buildings.iterrows(), total=len(buildings), desc="Finding nearest with index", unit="buildings"): 
    nearest_road = find_nearest_with_index(building.geometry, roads)
    building_road_analysis[building.gid] = {"nearest_road_id" : nearest_road.osm_id}

In [ ]:
building_road_analysis

#### Mini assignment

Find the building that is the furthest away from any road segment. Print the road name and the distance between the road and the building.

Tip 1: you probably want to use spatial indexing because you have to perform the analysis on the entire datasets..

Tip 2: you might want to adapt the function to be able to get the answer that is asked for..

<a id='case1'></a>
### Case 1 - Afstand-tot-Koelte: preprocessing the network
Our client has asked us to provide a heatstress analysis of their city. The client's wish is to get an overview of the distance from every building to the nearest cool spot.

To do this we will perform a network analysis (in `Part 2`). 

To prepare for our network analysis, our first task is to connect the buildings (polygons), roads (lines) and cool spots (polygons). We will connect the buildings by creating a line connecting the building with its nearest road segment. And we will do the same for the cool spots.

They provided us with three datasets, saved in the folder `material/part 1`.

| Geopackage - Layer                     | Description                                                                 |
|-----------------------------|-----------------------------------------------------------------------------|
| `Network Input - buildings` | Polygons representing the buildings |
| `Network Input - cool spots` | LineStrings representing the road network |
| `Network Input - roads` | Polygons representing the cool spots |

Follow these steps to set up the network:
1. Load the data into geodataframes.
2. Validate and improve the input data (make valid and explode).
3. Split the road geometries into small segments.
4. Simplify the polygon features.
5. Find the nearest road to each building and add a building connector (a line from building to road).
6. Find the nearest road to each cool spot and add a cool spot connector (a line from cool spot to road).
7. Combine the line layers (roads and connectors) to a line network.

You can make use of the functions introduced in the sections above. Additionally, you can make use of the functions provided below.

Once you get stuck, take a look at the answers to get you back on track.

In [ ]:
def split_line_geometries(gdf, max_segment_length):
    """ 
    returns geodataframe with splitted line geometries
    it uses split_line-by_length_2 for each line in the gdf
    """
    split_rows = []

    for idx, row in tqdm(gdf.iterrows(), total=len(gdf), desc="Choppin' them lines into lil' pieces..."):
        geometry = row['geometry']

        if isinstance(geometry, LineString) and geometry.length > max_segment_length:
            segments = split_line_by_length_2(geometry, max_segment_length)
            for segment in segments:
                new_row = row.copy()
                new_row['geometry'] = segment
                split_rows.append(new_row)
        else:
            split_rows.append(row)

    gdf = gpd.GeoDataFrame(split_rows, columns=gdf.columns, crs=gdf.crs)
    gdf.reset_index(drop=True, inplace=True)
    
    return gdf

In [ ]:
def update_geodataframe_with_sliced_geometries(gdf, threshold_perimeter):
    """
    returns geodataframe with sliced polygon geometries
    it uses slice_polygon_in_half for every polygon in the gdf with a perimeter >= threshold
    """
    new_rows = []
    indices_to_drop = []
    
    for index, row in gdf.iterrows():
        polygon = row['geometry']
        if polygon.exterior.length >= threshold_perimeter:
            polygon_halves = slice_polygon_in_half(polygon)
            
            # Create two new rows with the sliced polygons and add them to the list with original attributes
            new_rows.append({'geometry': polygon_halves[0], **row})
            new_rows.append({'geometry': polygon_halves[1], **row})

            # Drop the original row from the GeoDataFrame
            indices_to_drop.append(index)
    
    # Drop rows
    gdf.drop(indices_to_drop, inplace=True)
    
    # Append the new rows to the GeoDataFrame
    final_gdf = pd.concat([gdf, gpd.GeoDataFrame(new_rows)], ignore_index=True)
    
    return final_gdf

In [ ]:
def get_connecting_line(polygon_feature, line_feature):
    """
    Find the vertices of polygon_feature and line_feature that are the closest to each other.
    Create a LineString feature based on these vertices.
    """ 

    # Find the nearest point on the polygon's exterior to the midpoint of the line
    midpoint_line = line_feature.interpolate(0.5, normalized=True)
    closest_point_polygon = polygon_feature.exterior.interpolate(polygon_feature.exterior.project(midpoint_line))

    # Find the nearest point on the line to the point on the polygon
    closest_point_line = line_feature.interpolate(line_feature.project(closest_point_polygon))

    # Create a LineString feature based on the two closest points
    connecting_line = LineString([closest_point_polygon, closest_point_line])

    return connecting_line

In [ ]:
#1 Load data
path = Path.cwd().parent / "material" / "part 1" / "Network Input.gpkg"
buildings = gpd.read_file(path, layer='buildings')
cool_spots = gpd.read_file(path, layer='cool spots')
roads = gpd.read_file(path, layer='roads')

In [ ]:
#2a Make the invalid geometries valid
...

In [ ]:
#2b Explode the MultiPolygons into individual Polygons
...

In [ ]:
#3 Split roads
...

In [ ]:
#4 Simplify
...

In [ ]:
#5 Get connecting line between building and road, turn it into a gdf 
connecting_building_lines = []

for idx, building in tqdm(simplified_exploded_valid_buildings.iterrows(), 
                          total=len(simplified_exploded_valid_buildings), 
                          desc="Finding nearest building & getting connecting_line"
                          , unit="buildings"): 
    
    nearest_road = find_nearest_with_index(building.geometry, splitted_exploded_valid_roads)
    connecting_line = get_connecting_line(building.geometry, nearest_road.geometry)
    
    connecting_building_lines.append({
        "geometry" : connecting_line,
        "building_id" : building.gid,
        "road_id"     : nearest_road.id,
        "function" : "building_connector"
    })
    
connecting_building_lines_gdf = gpd.GeoDataFrame(connecting_building_lines, geometry='geometry', crs=buildings.crs)

In [ ]:
#6 Get connecting line between cool spot and road, turn it into a gdf
...

In [ ]:
#7 Combine to a network
splitted_exploded_valid_roads["function"] = "road" # add function to roads

network_gdf = pd.concat(
    [
     splitted_exploded_valid_roads,
     connecting_building_lines_gdf,
     connecting_cool_spot_lines_gdf
    ], 
    ignore_index=True
)

##### Finished?
After your done, write the results to a geopackage and check it out in QGIS. Write the following variables to the geopackage:
1. `network_gdf`
2. `sliced_simplified_exploded_valid_buildings`
3. `sliced_simplified_exploded_valid_cool_spots`

In [ ]:
folder_path = Path.cwd() / "output"
folder_path.mkdir(parents=True, exist_ok=True)

network_gdf.to_file(folder_path / "network_preprocessed.gpkg", driver="GPKG", layer="network preprocessed")
sliced_simplified_exploded_valid_buildings.to_file(folder_path / "network_preprocessed.gpkg", driver="GPKG", 
                                                   layer="buildings preprocessed")
sliced_simplified_exploded_valid_cool_spots.to_file(folder_path / "network_preprocessed.gpkg", driver="GPKG", 
                                                    layer="cool spots preprocessed")

#### Bonus
Look critically at the network. What could be improved and how would you do so? 

If you have the time and the skills, make your suggested improvements come to life.

...

<a id='bonus'></a>
### Bonus - Slicing polygons in half
A way of improving the network is by connecting the larger buildings to multiple points in the network. Let's start slicing polygons in half. 

Slicing a polygon in half can be quite the challenge. Polygons can have quite complex shapes and sizes. Nevertheless, it is worth to mention that all polygons are beautiful in their own way.

Use the following function:

In [ ]:
def slice_polygon_in_half(polygon):
    '''
    slice polygon in half by a vertical or horizontal line
    returns two halves (Polygon or MultiPolygon)
    '''
    min_x, min_y, max_x, max_y = polygon.bounds

    if (max_x - min_x) >= (max_y - min_y): # if width larger than height slice it by a vertical line
        mid_x = (min_x + max_x) / 2
        left_mask = Polygon([(min_x, min_y), (mid_x, min_y), (mid_x, max_y), (min_x, max_y)])
        right_mask = Polygon([(mid_x, min_y), (max_x, min_y), (max_x, max_y), (mid_x, max_y)])
               
        left_polygon = polygon.intersection(left_mask)
        right_polygon = polygon.intersection(right_mask)
        
        return [left_polygon, right_polygon]
    
    else: # slice it by a horizontal line
        mid_y = (min_y + max_y) / 2
        bottom_mask = Polygon([(min_x, min_y), (max_x, min_y), (max_x, mid_y), (min_x, mid_y)])
        top_mask = Polygon([(min_x, mid_y), (max_x, mid_y), (max_x, max_y), (min_x, max_y)])
        
        bottom_polygon = polygon.intersection(bottom_mask)
        top_polygon = polygon.intersection(top_mask)
        
        return [bottom_polygon, top_polygon]

Do you understand what the function does?

Let's run the function for large polygons in our dataset. Set the threshold perimeter to define a large polygon.

In [ ]:
threshold_perimeter = ...

sliced_polygons = {}
for idx, row in exploded_valid_polygons.iterrows():
    if row.geometry.exterior.length >= threshold_perimeter:
        polygon_halves = slice_polygon_in_half(row.geometry)
        sliced_polygons[row.geometry] = polygon_halves

Let's check out the sliced polygons by plotting them.

In [ ]:
def plot_polygon_with_halves(polygon, halves):
    polygon_x, polygon_y = polygon.exterior.xy
    half1_x, half1_y = halves[0].exterior.xy
    half2_x, half2_y = halves[1].exterior.xy

    fig, ax = plt.subplots()

    # Plot the original Polygon
    ax.fill(polygon_x, polygon_y, facecolor='none', edgecolor='black', linewidth=4, label='Original')

    # Plot the two halves on top
    ax.fill(half1_x, half1_y, facecolor='gold', label='Half 1')
    ax.fill(half2_x, half2_y, facecolor='teal', label='Half 2')

    ax.set_xlim(polygon.bounds[0], polygon.bounds[2])
    ax.set_ylim(polygon.bounds[1], polygon.bounds[3])
    ax.legend()

    plt.show()

In [ ]:
# Plot sliced polygons
for polygon, halves in sliced_polygons.items():
    plot_polygon_with_halves(polygon, halves)